In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions

# from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
# import setuptools.dist
import undetected_chromedriver as uc

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(2)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists DviraciaiVarle
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists DviraciaiVarle
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [3]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://www.varle.lt/dviraciai/?p=1'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [4]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'GRID_ITEM'})
for linkas in linkai:
    # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
    link = linkas.find('div', {'class':'product-title'}).find('a')
    # print(link['href'])
    add_link = 'https://www.varle.lt'+ link['href']
    # print(add_link)
    p_nuorodos.append(add_link)
print(len(p_nuorodos))
print(p_nuorodos)

pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)
print(pages_max_number)

38
['https://www.varle.lt/dviraciai/tunturi-competence-f20-treniruoklis--17564913.html', 'https://www.varle.lt/dviraciai/elektrinis-dviratis-kaisda-k7-20-350w-125ah--37472402.html', 'https://www.varle.lt/dviraciai/dviratis-esperia-226200u-life-comfort-54-tamsiai--19050839.html', 'https://www.varle.lt/dviraciai/el-dviratis-esperia-e960-50-36v128ah-2-juodageltona--19050852.html', 'https://www.varle.lt/dviratukai-triratukai/quurio-bike-vaikiskas-dviratis-quurio-robo-12--27780150.html', 'https://www.varle.lt/dviraciai/dviratis-azimut-city-lux-28-2023-light-melynas-baltas--40865748.html', 'https://www.varle.lt/dviratukai-triratukai/quurio-bike-vaikiskas-dviratis-quurio-wooohooo-12--28645665.html', 'https://www.varle.lt/dviratukai-triratukai/quurio-bike-vaikiskas-dviratis-quurio-yaaaaay-12--27432485.html', 'https://www.varle.lt/dviraciai/elektrinis-dviratis-hx-h1-juodas--24703908.html', 'https://www.varle.lt/dviratukai-triratukai/quurio-bike-vaikiskas-dviratis-quurio-sweeeeet-12--27796552.ht

In [5]:
for nuoroda in p_nuorodos[1:2]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
    url = nuoroda
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.quit()

https://www.varle.lt/dviraciai/elektrinis-dviratis-kaisda-k7-20-350w-125ah--37472402.html


In [6]:
bs = BeautifulSoup(source, 'html.parser')
pavadinimas = bs.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
print(pavadinimas)

# varle_kodas = bs.find('div', {'class':'main'}).find('span', {'itemprop':'sku'}).text
# print(varle_kodas)

kaina = bs.find('span', {'class':'price-value'}).text.strip()
print(kaina)

buvusi_kaina = bs.find('span', {'class':'previous-price'})
if buvusi_kaina != None:
    buvusi_kaina = buvusi_kaina.text.strip()
else:
    buvusi_kaina = None
print(buvusi_kaina)

specifiacija_keys = []
specifiacija_values = []
try:
    specifiacija = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
    for line in specifiacija:
        try:
            spec_key = line.find('span', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('span', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
        try:
            spec_key = line.find('div', {'class':'left-column'}).text
            # print(spec_key)
            specifiacija_keys.append(spec_key)
            spec_value = line.find('div', {'class':'right-column'}).text
            # print(spec_value)
            specifiacija_values.append(spec_value)
        except:
            pass
except:
    pass

try:
    specifiacija_k = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
    for key in specifiacija_k:
        # print(key.text.strip())
        specifiacija_keys.append(key.text.strip().lower())
    specifiacija_v = bs.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
    for val in specifiacija_v:
        # print(val.text.strip())
        specifiacija_values.append(val.text.strip())       
except:
    pass

print(len(specifiacija_keys), specifiacija_keys)
print(len(specifiacija_values), specifiacija_values)

spec_filtrai_keys = []
spec_filtrai_values = []
try:
    spec_filtai = bs.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
    for line in spec_filtai:
        spec_key = line.find('span', {'class':'left-column'}).text.strip().lower()
        # print(spec_key)
        spec_filtrai_keys.append(spec_key)
        spec_value = line.find('span', {'class':'right-column'}).text.strip()
        # print(spec_value)
        spec_filtrai_values.append(spec_value)
except:
    pass
    
print(len(spec_filtrai_keys), spec_filtrai_keys)
print(len(spec_filtrai_values), spec_filtrai_values)

# for k,v in zip(specifiacija_keys, specifiacija_values):
#     print(k, v)

Elektrinis dviratis Kaisda K7, 20", 350W, 12,5Ah, baltas
799.99
1099.99
 €
4 ['Gamintojas', 'Modelis', 'Stabdžiai', 'Svoris']
4 ['Kaisda', 'K7', 'Diskiniai', '25']
4 ['gamintojas', 'tipas', 'spalva', 'variklio galingumas (w)']
4 ['Kaisda', 'Elektriniai', 'Balta', '350']


In [ ]:
# createMainTable('VarlePigu.db')


True

In [ ]:
# Dviraciai Varle scraper

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
# driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dviraciai/?p=9'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
pages = []
pages_numbers = bs.find('div', {'class':'PAGINATION'}).find_all('li', {'class':'number'})
for page in pages_numbers:
    # print(page.text.strip())
    pages.append(int(page.text.strip()))
pages_max_number = max(pages)

counts = 306
psl = 9
iki_kokio_psl = 12
while psl < iki_kokio_psl:
# while psl < pages_max_number + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []

    linkai = bs.find_all('div', {'class':'GRID_ITEM'})
    for linkas in linkai:
        # linkas = bs.find('div', {'class':'product-title'}).find('div', {'class':'product-title'}).find('a')
        link = linkas.find('div', {'class':'product-title'}).find('a')
        # print(link['href'])
        add_link = 'https://www.varle.lt'+ link['href']
        # print(add_link)
        p_nuorodos.append(add_link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        time.sleep(random.randint(5,10))
        # time.sleep(4)
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='DviraciaiVarle', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs_preke = BeautifulSoup(source, 'html.parser')
        
        firstInfo = bs_preke.find('div', {'class':'main'}).find('h1', {'class':'title'}).text.strip()
        duomenys['firstInfo'] = firstInfo
        # print(pavadinimas)


        kaina = bs_preke.find('span', {'class':'price-value'}).text.strip()
        duomenys['kaina'] = kaina
        # print(kaina)

        buvusi_kaina = bs_preke.find('span', {'class':'previous-price'})
        if buvusi_kaina != None:
            buvusi_kaina = buvusi_kaina.text.strip()
        else:
            buvusi_kaina = None
        duomenys['buvusiKaina'] = buvusi_kaina
        # print(buvusi_kaina)

        specifiacija_keys = []
        specifiacija_values = []
        try:
            specifiacija = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('li', {'class':'spec-line'})
            for line in specifiacija:
                try:
                    spec_key = line.find('span', {'class':'left-column'}).text.lower()
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('span', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
                try:
                    spec_key = line.find('div', {'class':'left-column'}).text.lower()
                    # print(spec_key)
                    specifiacija_keys.append(spec_key)
                    spec_value = line.find('div', {'class':'right-column'}).text
                    # print(spec_value)
                    specifiacija_values.append(spec_value)
                except:
                    pass
        except:
            pass

        try:
            specifiacija_k = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'key'})
            for key in specifiacija_k:
                # print(key.text.strip())
                specifiacija_keys.append(key.text.strip().lower())
            specifiacija_v = bs_preke.find('div', {'class':'PRODUCT_SPEC'}).find_all('td', {'class':'value'})
            for val in specifiacija_v:
                # print(val.text.strip())
                specifiacija_values.append(val.text.strip())       
        except:
            pass
        
        
        # print(len(specifiacija_keys), specifiacija_keys)
        # print(len(specifiacija_values), specifiacija_values)

        spec_filtrai_keys = []
        spec_filtrai_values = []
        try:
            spec_filtai = bs_preke.find('div', {'class':'PRODUCT_SPEC_FILTERS'}).find_all('li', {'class':'spec-line'})
            for line in spec_filtai:
                spec_key = line.find('span', {'class':'left-column'}).text.strip().lower()
                # print(spec_key)
                spec_filtrai_keys.append(spec_key)
                spec_value = line.find('span', {'class':'right-column'}).text.strip()
                # print(spec_value)
                spec_filtrai_values.append(spec_value)
                
            # print(len(spec_filtrai_keys), spec_filtrai_keys)
            # print(len(spec_filtrai_values), spec_filtrai_values)
        except:
            pass
        
        for k, v in zip(spec_filtrai_keys, spec_filtrai_values):
            duomenys[k] = v
            
        for k, v in zip(specifiacija_keys, specifiacija_values):
            duomenys[k] = v
            
        cols_i_need = list(duomenys.keys())
        
        data = tuple(duomenys.values())
        
        cols_i_have = getColNamesList(table='DviraciaiVarle', dbname='VarlePigu.db')
        
        alterTableAddColumn(cols_i_need, cols_i_have, 'DviraciaiVarle', 'VarlePigu.db')
        
        writeMany(data, 'DviraciaiVarle', 'VarlePigu.db')
        print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
        counts = counts +1
    
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    if psl < iki_kokio_psl:
    # if psl <= pages_max_number:
        next_nuoroda = f'https://www.varle.lt/dviraciai/?p={psl}'
        driver.get(next_nuoroda)
        time.sleep(5)
        print(f'kitas puslapis: {psl}')
    
driver.quit()
print('Baigta!!!')


6 https://www.varle.lt/dviraciai/dviratis-romet-rambler-fit-26-2024-juodas-melynas-18--33192162.html
Įrašytas įrašas Nr 191, puslapis 6
6 https://www.varle.lt/dviraciai/dviratis-romet-rambler-r63-2024-melynas-red-sidabrinis-18--31839288.html
Įrašytas įrašas Nr 192, puslapis 6
6 https://www.varle.lt/dviraciai/dviratis-romet-wagant-0-2024-bright-graphite-zalias-23--31839302.html
Įrašytas įrašas Nr 193, puslapis 6
6 https://www.varle.lt/dviraciai/dviratis-merida-bigseven-15-i1-matt-dark-sidabrinissidabrinis-38--32867564.html
Įrašytas įrašas Nr 194, puslapis 6
6 https://www.varle.lt/dviraciai/dviratis-merida-bigseven-15-i1-matt-dark-sidabrinissidabrinis-43--32867566.html
Įrašytas įrašas Nr 195, puslapis 6
6 https://www.varle.lt/dviraciai/dviratis-romet-pop-art-eco-28-alu-2022-violet-20--17826390.html
Įrašytas įrašas Nr 196, puslapis 6
6 https://www.varle.lt/dviraciai/dviratis-romet-sonata-classic-28-2024-baltas-18--32852081.html
Įrašytas įrašas Nr 197, puslapis 6
6 https://www.varle.lt/dvi